In [ ]:
import numpy as np
import holoviews as hv
from numpy import exp, diag

hv.notebook_extension()
hv.extension("bokeh")

In [ ]:
def u_exact(x, eps):
    return (exp(x / eps) - exp(1 / eps)) / (1 - exp(1 / eps))


In [ ]:
def Af(eps, N):
    h = 1 / N
    R = diag([ -eps - h] * N   , -1) + \
        diag([2*eps + h] *(N+1),  0) + \
        diag([ -eps    ] * N   ,  1)
    
    R = R / h**2
    
    R[0,:] = 0 # make the top row all zero
    R[0,0] = 1 # make the left-top cell 1
    R[N,:] = 0 # bottom row = 0
    R[N,N] = 1 # right-bottom cell = 1
    
    f = [1] + [0]*N # make the vector f

    return np.array(R, dtype=float), np.array(f, dtype=float) # do some conversion to socalled numpy objects

Af(.5, N=10)

In [ ]:
# %%opts Curve  [height=600 width=600 tools=['hover']]

N = 10
eps = 0.5

A,f = Af(eps, N)
ys = np.linalg.solve(A,f)
xs = np.linspace(0,1,N+1)
print(ys)

xx = np.linspace(0, 1)
(hv.Scatter((xs,ys)).relabel("numerical").options(color="red") * 
 hv.Curve((xx, u_exact(xx, eps))).relabel("exact"))

In [ ]:
# %%opts Curve  [height=600 width=600 tools=['hover']]
# %%opts Scatter (marker='x' size=10)
# exercise 1

plot = hv.Scatter(([],[])) * hv.Curve(([],[]))
for eps in [1, 0.5, 0.2, 0.05, 0.01]:
    N = 15

    A,f = Af(eps, N)
    ys = np.linalg.solve(A,f)
    xs = np.linspace(0,1,N+1)

    xx = np.linspace(0, 1, 200)
    plot *= hv.Curve((xx, u_exact(xx, eps))).relabel(f"eps = {eps}")
    plot *= hv.Scatter((xs,ys)).relabel(f"eps = {eps}")
            

plot.options(legend_position="top")

In [ ]:
# exercise 2

plot = hv.Scatter(([],[])) * hv.Curve(([],[]))
table = []
for eps in [1, 0.5, 0.2, 0.05, 0.01]:
    row = []
    for N in [16, 32, 64, 128, 256, 512, 1024]:
        A,f = Af(eps, N)
        ys = np.linalg.solve(A,f)
        xs = np.linspace(0,1,N+1)

        y_ex = u_exact(xs, eps)
        
        error = np.max(np.abs(ys - y_ex))
        h = 1/N
        print(f"eps={eps:.4f}, N = {N:5d}, error = {error:.4f}, err/h = {error/h:.4f}")
        row.append(error)
    print()
    table.append(row)
np.array(table)

In [ ]:
# opgave 3

# not symmetrix

A,f = Af(eps=1, N=5)
np.linalg.inv(A)

In [ ]:
# %%opts Curve  [height=600 width=600 tools=['hover']]
# %%opts Scatter (marker='x' size=3)
# exercise 4

# for N in [8, 16, 32]: #, 32, 64, 128, 256, 512, 1024]:
N = 128
eps = .2

plot = hv.Scatter(([],[]))
eigenvector_plot = hv.Scatter(([],[]))

A,f = Af(eps, N)
plot *= hv.Scatter(sorted(np.linalg.eigvals(A))).relabel(f"{eps}")

eig, vv = np.linalg.eig(A)

idx = eig.argsort()  
eig = eig[idx]
vv = vv[:,idx]

plot = hv.Scatter(eig).relabel(f"{eps}")
plot.options(legend_position="top")

In [ ]:
hv.Curve(vv[:,0]) + hv.Curve(vv[:,1]) + hv.Curve(vv[:,15])+ hv.Curve(vv[:,64])+ hv.Curve(vv[:,127])

In [ ]:
np.linalg.eigvals(A)

In [ ]:
# exercise 5

table = []
for eps in [1, 0.5, 0.2, 0.05, 0.01]:
    row = []
    for N in [8, 16, 32, 64, 128, 256, 512]:
        A,f = Af(eps, N)
        D = np.diag(np.diag(A))
        B = np.identity(N+1) - np.linalg.inv(D) @ A
        
        eigs = np.linalg.eigvals(B)
        rho = np.max(eigs)
        
        print(f"eps={eps:.4f}, N = {N:5d}, rho = {rho:.4f}")
    print()


In [ ]:
# %%opts Scatter (marker='x' size=3)
eps = 0.5

plot = hv.Scatter([])

for N in [8, 16, 32, 64]:
    u = [0] * (N+1)
    A, f = Af(N=N, eps=eps)
    n = 0
    residuals = []
    while True:
        n += 1
        # single step
        z = u[:] # copy of u
        for i in range(N+1): # i = 0 .. N
            z[i] = (f[i] - A[i,0:i] @ u[0:i] - A[i,i+1:N] @ u[i+1:N]) / A[i, i]
        u = z

        r = f - A @ u
        stopping_crit = np.max(np.abs(r)) / np.max(np.abs(f))
        residuals.append(stopping_crit)

        if stopping_crit < 1e-6:
            plot *= hv.Scatter(residuals).relabel(f"N={N}")
            print(N)
            print(residuals[-5] / residuals[-6])
            print(residuals[-4] / residuals[-5])
            print(residuals[-3] / residuals[-4])
            print(residuals[-2] / residuals[-3])
            print(residuals[-1] / residuals[-2])
            break

plot.options(legend_position="top", logy=True)

In [ ]:
# 3 * N**2

(0.948 + 0.888) / 2